In [1]:
import PipelineERVs as PE
reload(PE)
import pandas as pd

In [4]:
tab = pd.read_csv("/ifs/projects/katherineb/ERVS_test/envs2_results.tsv", sep="\t", index_col=0)
tab['target_start'] = tab['target_start'].astype(int)
tab['target_end'] = tab['target_end'].astype(int)
D = dict()
with open("/ifs/projects/katherineb/ERVS_test/envs_merged.bed") as infile:
    for line in infile:
        line = line.strip().split("\t")
        if line[0] not in D:
            D[line[0]] = []
        D[line[0]].append((int(line[1]), int(line[2])))
cnames = tab.columns.tolist()
cnames.append('final_target_start')
cnames.append('final_target_end')
cnames.append('final_length')

results = pd.DataFrame(columns=cnames)
for chromosome in D:
    subtab = tab[(tab['target_id'] == chromosome)]
    for pos_pair in D[chromosome]:
        overlappers = subtab[((subtab['target_start'] >= pos_pair[0]) &
                              (subtab['target_end'] <= pos_pair[1]))]
        scores = overlappers['score'].astype(int)
        best = scores[scores == max(scores)]
        result = overlappers.ix[best.index]
        result['final_target_start'] = pos_pair[0]
        result['final_target_end'] = pos_pair[1]
        result['final_length'] = pos_pair[1] - pos_pair[0]
        results = results.append(result)
results = results[cnames]

In [8]:
results.values

array([['Baboon_ERV_D10032_beta_env', 444.0, 543.0, '.', 'chr21',
        17765117.0, 17765417.0, '-', 187.0,
        'M|49|147|G|0|6|M|5|15|G|1|0|M|44|132', 300.0, 17765102.0,
        17765417.0, 315.0],
       ['Baboon_ERV_D10032_beta_env', 307.0, 537.0, '.', 'chr21',
        18677078.0, 18677760.0, '+', 381.0,
        'M|28|84|F|0|4|M|34|102|G|2|0|M|123|369|G|2|0|M|41|123', 682.0,
        18677078.0, 18677760.0, 682.0],
       ['Mouse_mammary_tumour_virus_NC_001503_beta_env', 395.0, 521.0, '.',
        'chr21', 19934375.0, 19934759.0, '-', 134.0,
        'M|15|45|G|0|3|M|31|93|G|0|3|M|80|240', 384.0, 19934375.0,
        19934759.0, 384.0],
       ['Baboon_ERV_D10032_beta_env', 437.0, 550.0, '.', 'chr21',
        28233036.0, 28233373.0, '+', 232.0,
        'M|51|153|F|0|1|M|34|102|G|1|0|M|27|81', 337.0, 28233036.0,
        28233373.0, 337.0],
       ['Simian_ERV_JN134185_beta_env', 375.0, 535.0, '.', 'chr21',
        30728044.0, 30728519.0, '-', 313.0,
        'M|120|360|G|2|0|M|3|9|

In [ ]:
results

In [ ]:
cnames = ["query_id", "query_start", "query_end", "query_strand", "target_id", "target_start", "target_end",
          "target_strand", "score", "details"]
df = pd.DataFrame(columns=cnames)


In [ ]:
with open("/ifs/projects/katherineb/ERVS_test/raw_exonerate_output/envs_chr21.tsv") as infile:
    for line in infile:
        line = line.split(": ")
        if line[0] == "vulgar":
            segs = line[1].split(" ")
            for i in range(9):
                df.ix[segs[0], cnames[i]] = segs[i]
            rest = "|".join(line[9:])
            df.ix[segs[0], cnames[9]] = rest

In [ ]:
    with open(infile) as infile:
        for line in infile:
            line = line.strip().split(":")


In [ ]:
df['target_start'] = df['target_start'].astype(int)

In [ ]:
df['target_end'] = df['target_end'].astype(int)

In [ ]:
    outs = ['/ifs/projects/katherineb/ERVS_test/raw_exonerate_output/envs2_chr14.tsv', '/ifs/projects/katherineb/ERVS_test/raw_exonerate_output/envs2_chr21.tsv']
    cnames = ["query_id", "query_start", "query_end", "query_strand",
              "target_id", "target_start", "target_end",
              "target_strand", "score", "details"]
    j = 0
    bigdf = pd.DataFrame(columns=cnames)
    
    for out in outs:
        smalldf = pd.DataFrame(columns=cnames)
        with open(out) as infile:
            for line in infile:
                line = line.split(": ")
                if line[0] == "vulgar":
                    segs = line[1].split(" ")
                    for i in range(9):
                        smalldf.ix[str(j), cnames[i]] = segs[i]
                    rest = "|".join(segs[9:])
                    smalldf.ix[str(j), cnames[9]] = rest
                    j += 1
        smalldf = smalldf.sort(['target_start'])
        bigdf = bigdf.append(smalldf)




In [ ]:
bigdf['target_start'] = bigdf['target_start'].astype(int)
bigdf['target_end'] = bigdf['target_end'].astype(int)

In [ ]:
bigdf['Length'] = abs(bigdf['target_end'] - bigdf['target_start'])

In [ ]:
bigdf  = bigdf[bigdf['details'].str.contains("I") == False]

In [ ]:
bigdf[bigdf['Length'] > 200]

In [ ]:
r1 = bigdf.ix['33']

In [ ]:
r2 = bigdf.ix['65']

In [ ]:
def isOverlapping(region1, region2):
    r1 = (min(region1['target_start'], region1['target_end']),
               max(region1['target_start'], region1['target_end']))
    r2 = (min(region2['target_start'], region2['target_end']),
               max(region2['target_start'], region2['target_end']))
    case1 = r1[0] < r2[0]
    

In [ ]:
isOverlapping(r1, r2)

In [ ]:
df2 = pd.read_csv('/ifs/projects/katherineb/ERVS_test/raw_exonerate_output/envs2.tsv', sep="\t", index_col=0)

In [ ]:
df2 = df2[((df2.Length>200) & (df2.details.str.contains("I") == False))]


In [ ]:
df2[cnames]

In [ ]:
plus = smalldf[smalldf['target_start'] < smalldf['target_end']]
minus = smalldf[smalldf['target_start'] > smalldf['target_end']]

In [ ]:
tempstart = minus['target_start']
tempend = minus['target_end']

In [ ]:
minus = minus.drop(['target_start', 'target_end'], 1)

In [ ]:
minus['target_start'] = tempend
minus['target_end'] = tempstart

In [ ]:
reload(PE)
x = PE.filterExonerate("/ifs/projects/katherineb/ERVS_test/raw_exonerate_output/envs2_chr21.tsv", 200)

In [ ]:
x['target_start'] = x['target_start'].astype('object')

In [ ]:
x